In [1]:
import os

In [5]:
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

In [6]:
deployment_name = os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"]
endpoint = os.environ["AZURE_OPENAI_ENDPOINT"]
api_key = os.environ["AZURE_OPENAI_API_KEY"]

In [11]:
import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion

In [9]:
kernel = sk.Kernel()

In [12]:
kernel.add_chat_service("dv", AzureChatCompletion(
    deployment_name=deployment_name,
    endpoint=endpoint,
    api_key=api_key)
)

In [28]:
prompt = kernel.create_semantic_function("""
I need to understand what are the variables involved in making outstanding espresso besides
a good machine. For example what is the combination of roast, grind, tamp, and water temperature.
Include 3 practical steps to practice and improve each variable.
""")

# Run your prompt
print(prompt())

There are several variables involved in making outstanding espresso, including:

1. Roast: The roast level of the coffee beans can greatly affect the flavor of the espresso. Lighter roasts tend to have more acidity and fruitiness, while darker roasts have more bitterness and chocolate notes.

Practical steps to practice and improve roast:

- Experiment with different roast levels to find the one that suits your taste preferences.
- Keep track of the roast level of the beans you use and note how it affects the flavor of your espresso.
- Try different blends of beans to find the perfect combination for your taste.

2. Grind: The grind size of the coffee beans is crucial in determining the extraction rate and flavor of the espresso. A finer grind will result in a slower extraction and a stronger, more intense flavor, while a coarser grind will result in a faster extraction and a weaker, less intense flavor.

Practical steps to practice and improve grind:

- Invest in a high-quality grinde

In [17]:
from semantic_kernel.skill_definition import sk_function, sk_function_context_parameter
from semantic_kernel.orchestration.sk_context import SKContext

In [47]:
class TravelWeather:
    @sk_function(
        description="Takes a city and a month and returns the average temperature for that month.",
        name="travel_weather",
    )
    @sk_function_context_parameter(
        name="city", description="The city for which to get the average temperature."
    )
    @sk_function_context_parameter(
        name="month", description="The month for which to get the average temperature."
    )
    def weather(self, context: SKContext) -> str:
        return f"The average temperature in city in month is 75 degrees. {str(context.variables)}"

        city = context["city"]
        month = context["month"]
        return f"The average temperature in {city} in {month} is 75 degrees."

In [48]:
weather_plugin = kernel.import_skill(TravelWeather(), skill_name="Travel")

In [49]:
prompt_config = sk.PromptTemplateConfig.from_completion_parameters(
    max_tokens=200,
    temperature=0.7,
    top_p=0.8,
    function_call="auto",
    chat_system_prompt="You are a travel weather chat bot. Your name is Frederick. You are trying to help people find the average temperature in a city in a month.",
)

In [50]:
prompt_template = sk.ChatPromptTemplate(
    "{{$user_input}}", kernel.prompt_template_engine, prompt_config
)
prompt_template.add_user_message("Hi there, who are you?")
prompt_template.add_assistant_message(
    "I am Frederic, a chat bot. I'm trying to figure out what people need."
)

In [51]:
function_config = sk.SemanticFunctionConfig(prompt_config, prompt_template)
chat_function = kernel.register_semantic_function("ChatBot", "Chat", function_config)
functions = [
    {
        "name": "travel_weather",
        "description": "Finds the average temperature for a city in a month.",
        "parameters": {
            "type": "object",
            "properties": {
                "city": {
                    "type": "string",
                    "description": "The city for example Madrid",
                },
                "month": {
                    "type": "string",
                    "description": "The month of the year, for example June",
                },
            },
            "required": ["city", "month"],
        },
    }
]

In [52]:
context = kernel.create_new_context()

In [53]:
context.variables["user_input"] = "What is the average temperature in Seattle in June?"
context = await chat_function.invoke_async(context=context, functions=functions)

In [43]:
chat_function._chat_prompt_template.messages.append({"role": "assistant", "content": "It is 85 degrees in Seattle in June"})
context = await chat_function.invoke_async(context=context, functions=functions)

In [54]:
print("No function was called")
print(f"Output was: {str(context)}")

No function was called
Output was: 
